<a href="https://colab.research.google.com/github/rubyvanrooyen/NIFTyworkshop/blob/master/NIFTy_Example_RMsynth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -b NIFTy_6 --single-branch https://gitlab.mpcdf.mpg.de/ift/nifty.git
!pip install ./nifty

In [ ]:
import nifty6 as ift
import numpy
import numpy as np
from matplotlib import pylab

In [ ]:
def p_lambda2(x, p0 = 0.5, RM=0., DRM=500., fa=12.):
    PA = numpy.pi/fa
    p = p0 \
    * (numpy.sin(DRM * x )/(DRM*x) ) \
    * numpy.exp(2j* RM * x) \
    * numpy.exp(2j * PA)
    return p

def faraday_synthesis(x, pol, phi_range, lambda0=False):
    f = numpy.zeros(len(phi_range), dtype=complex)

    N = len(x)
    xmean = x.mean()

    for k, phi in enumerate(phi_range):
        if lambda0:
            f[k] = numpy.sum(pol * numpy.exp(-2j * phi * (x-xmean) ))/N
        else:
            f[k] = numpy.sum(pol * numpy.exp(-2j * phi * x ))/N
                    
    return f

# define lambda^2
dx = 3.835e-5 # m^2 
x = numpy.arange((3.0e8/18e9)**2, (3.0e8/2e9)**2, dx)
# define Faraday depth range
phi_range = numpy.arange(-10000, 10010, 10)

p = p_lambda2(x)

f = faraday_synthesis(x,  # wavelength: wavelength squared.
                      p,
                      phi_range,  # Faraday depth range
                      lambda0=False)

In [ ]:
Npixels=len(f)
# One-dimensional regular grid
position_space = ift.RGSpace([Npixels])
mask = np.zeros(position_space.shape)

# Specify harmonic space corresponding to signal
harmonic_space = position_space.get_default_codomain()

# Harmonic transform from harmonic space to position space
HT = ift.HarmonicTransformOperator(harmonic_space, target=position_space)

# 1D spectral space on which the power spectrum is defined
power_space = ift.PowerSpace(harmonic_space)

# Mapping to (higher dimensional) harmonic space
PD = ift.PowerDistributor(harmonic_space, power_space)

# Apply the mapping
# Set prior correlation covariance with a power spectrum leading to
# homogeneous and isotropic statistics
def power_spectrum(k):
    return 100./(20. + k**3)

prior_correlation_structure = PD(ift.PS_field(power_space, power_spectrum))

# Insert the result into the diagonal of an harmonic space operator
S = ift.DiagonalOperator(prior_correlation_structure)
# S is the prior field covariance

# Build instrument response consisting of a discretization, mask
# and harmonic transformaion

# Masking operator to model that parts of the field have not been observed
mask = ift.Field.from_raw(position_space, mask)
Mask = ift.MaskOperator(mask)

# The response operator consists of
# - a harmonic transform (to get to image space)
# - the application of the mask
# - the removal of geometric information
# The removal of geometric information is included in the MaskOperator
# it can also be implemented with a GeometryRemover
# Operators can be composed either with parenthesis
R = Mask(HT)
data_space = R.target

# Set the noise covariance N
noise = 0.2
N = ift.ScalingOperator(data_space, noise)

# Define test signal (some point sources)
signal = ift.makeField(data_space, np.abs(f))

# Build inverse propagator D and information source j
D_inv = R.adjoint @ N.inverse @ R + S.inverse
j = R.adjoint_times(N.inverse_times(signal))
# Make D_inv invertible (via Conjugate Gradient)
IC = ift.GradientNormController(iteration_limit=500, tol_abs_gradnorm=1e-3)
D = ift.InversionEnabler(D_inv, IC, approximation=S.inverse).inverse

# Calculate WIENER FILTER solution
m = D(j)

In [ ]:
# Get signal data and reconstruction data
# R(m) posterior mean -- recontructed signal
m_data = R(m).val
# corrupt measured signal
d_data = signal.val

# recontruct and compare
fig, ax = pylab.subplots(1, 1, figsize=(15, 3), facecolor='white')
ax.plot(d_data, 'k.', label="Data")
ax.plot(m_data, 'k', label="Reconstruction",linewidth=3)
ax.set_title("Reconstruction")
pylab.legend()
pylab.show()